## 机器学习进阶（S13）毕业项目 - 猫狗大战

## 项目：识别图片中是猫还是狗
---
* 此项目是kaggle平台上的一个比赛项目，最终的要求是提供一个模型来识别图片中的对象是猫还是狗。
* 这里会使用深度学习中的卷积神经网络来构建模型。
* 最总要求测试评分进入 kaggle Public Leaderboard 前10%。

## 步骤 1: 数据预处理

#### 载入数据

In [1]:
# 读取训练集
import os
import shutil

train_files = os.listdir('train')
print(train_files[0:10])

['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.1000.jpg', 'cat.10000.jpg', 'cat.10001.jpg', 'cat.10002.jpg', 'cat.10003.jpg', 'cat.10004.jpg']


#### 寻找异常

In [2]:
# 定义ImageNet中识别猫狗的分类 https://blog.csdn.net/zhangjunbob/article/details/53258524
dogs = [
 'n02085620','n02085782','n02085936','n02086079','n02086240','n02086646','n02086910','n02087046'
,'n02087394','n02088094','n02088238','n02088364','n02088466','n02088632','n02089078','n02089867'
,'n02089973','n02090379','n02090622','n02090721','n02091032','n02091134','n02091244','n02091467'
,'n02091635','n02091831','n02092002','n02092339','n02093256','n02093428','n02093647','n02093754'
,'n02093859','n02093991','n02094114','n02094258','n02094433','n02095314','n02095570','n02095889'
,'n02096051','n02096177','n02096294','n02096437','n02096585','n02097047','n02097130','n02097209'
,'n02097298','n02097474','n02097658','n02098105','n02098286','n02098413','n02099267','n02099429'
,'n02099601','n02099712','n02099849','n02100236','n02100583','n02100735','n02100877','n02101006'
,'n02101388','n02101556','n02102040','n02102177','n02102318','n02102480','n02102973','n02104029'
,'n02104365','n02105056','n02105162','n02105251','n02105412','n02105505','n02105641','n02105855'
,'n02106030','n02106166','n02106382','n02106550','n02106662','n02107142','n02107312','n02107574'
,'n02107683','n02107908','n02108000','n02108089','n02108422','n02108551','n02108915','n02109047'
,'n02109525','n02109961','n02110063','n02110185','n02110341','n02110627','n02110806','n02110958'
,'n02111129','n02111277','n02111500','n02111889','n02112018','n02112137','n02112350','n02112706'
,'n02113023','n02113186','n02113624','n02113712','n02113799','n02113978']

cats=['n02123045','n02123159','n02123394','n02123597','n02124075','n02125311','n02127052']


# 采用InceptionResNetV2预测
# 参考 https://keras.io/zh/applications/#applications
# from keras.models import *
# from keras.layers import *
# from keras.applications import *
# from keras.preprocessing.image import *
# import numpy as np

#存储不能识别为猫狗的异常图片
my_unknow_images = []

# # 输入：图片路径
# # 输出：预测概率，格式(class, description, probability)
# #       例如 [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357)]
def my_predict(img_path_list):
    model = InceptionResNetV2(weights='imagenet')
    for image_path in img_path_list:
        img = image.load_img(img_path, target_size=(299, 299))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        preds = model.predict(x)
        result = decode_predictions(preds, top=10)[0]
        if not is_cat_dog(result):
            my_unknow_images.append(image_path)

# 根据模型预测的结果查看是否属于猫狗的范围
def is_cat_dog(preds_result):
    my_labels = dogs[:]
    my_labels[len(my_labels):len(my_labels)] = cats
    a = [x[0] in my_labels for x in preds_result]
#     print(sum(a))
    return sum(a) > 0        

# iscatdog = is_cat_dog([(u'n02504013', u'Indian_elephant', 0.82658225), 
#                        (u'n01871265', u'tusker', 0.1122357), 
#                        (u'n02123394', u'test', 0.0000000)])
# print(iscatdog)

# my_predict(['train/cat.0.jpg'])
print(my_unknow_images)

[]


#### 显示异常图片

In [3]:
plt.figure(figsize=(16, 16))
plt.title('aaa')
plt.axis('off')
plt.imshow('train/cat.0.jpg')

NameError: name 'plt' is not defined

#### 处理异常图片

In [4]:
# 把异常图片从train移动到unknown文件夹
def move_images_to(files, dest_path):
    if not os.path.exists(dest_path):
        os.mkdir(dest_path)
        
    for file in files:
        shutil.move("t1/" + file, dest_path)
    
# files = ['cat.3.jpg','cat.5.jpg']
# move_images_to(files, 'unknow')


#### 细分训练数据

## 步骤 2: 模型搭建

In [15]:
from operator import is_not
from functools import partial
train_files = os.listdir('train')
train_cat_list = [file if file.split('.')[0] == 'cat' else None for file in train_files]
train_cat_list = list(filter(None.__ne__, train_cat_list))
# train_cat_list = filter(partial(is_not, None), train_cat_list)

train_dog_list = [file if file.split('.')[0] == 'dog' else None for file in train_files]
train_dog_list = list(filter(None.__ne__, train_dog_list))
# train_dog_list = filter(partial(is_not, None), train_dog_list)

print('train cat count:', len(train_cat_list))
print('train dog count:', len(train_dog_list))

print(train_cat_list[0:10])
print(train_dog_list[0:10])

def my_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

my_mkdir('train_classify')
my_mkdir('train_classify/cat')
my_mkdir('train_classify/dog')

# my_mkdir('test_kaggle')
# os.symlink('../test/', 'test_kaggle/test')

for file in train_cat_list:
    os.symlink('train/' + file, 'train_classify/cat/' + file)
    
for file in train_dog_list:
    os.symlink('train/' + file, 'train_classify/dog/' + file)

# for x in train_cat_list:
#     print(x)
# for x in train_dog_list:
#     print(x)



train cat count: 12500
train dog count: 12500
['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.1000.jpg', 'cat.10000.jpg', 'cat.10001.jpg', 'cat.10002.jpg', 'cat.10003.jpg', 'cat.10004.jpg']
['dog.0.jpg', 'dog.1.jpg', 'dog.10.jpg', 'dog.100.jpg', 'dog.1000.jpg', 'dog.10000.jpg', 'dog.10001.jpg', 'dog.10002.jpg', 'dog.10003.jpg', 'dog.10004.jpg']


## 步骤 3: 模型训练

## 步骤 4: 模型调参

## 步骤 5: 模型评估

### 预测测试集

## 步骤 6: 可视化